In [1]:
# Siamese Network with Triplet Loss Function for Face Recognition

# This file:
# 1) Defines the triplet loss function to be used to calculate loss (cost) during training
# 2) Loads the pretrained ResNet50 model from Keras and modifies it to create the Siamese Network
# 3) Trains the siamese network based on triplets available in the train set
# 4) After training, saves the original Siamese Network (that takes 3 inputs and gives 1 output (loss))
# 5) Also saves a separate model that just takes 1 img file and generates the face encoding of this input img.
#    This model is to be used during prediction

# Note that evaluation on dev set is done in a separate file.

In [1]:
#import tensorflow.contrib.eager as tfe
import tensorflow as tf
#from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
#import PIL
#from PIL import Image
from keras.models import Model
from keras.layers import Dense, Input, subtract, concatenate, Lambda, add, maximum
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras.models import load_model, model_from_json
import numpy as np

/home/cesncn/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# DO NOT USE EAGER MODE
#tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.8.0
Eager execution: False


In [3]:
def triplet_loss(inputs, dist='euclidean', margin='maxplus'):
    anchor, positive, negative = inputs
    positive_distance = K.square(anchor - positive)
    negative_distance = K.square(anchor - negative)
    if dist == 'euclidean':
        positive_distance = K.sqrt(K.sum(positive_distance, axis=-1, keepdims=True))
        negative_distance = K.sqrt(K.sum(negative_distance, axis=-1, keepdims=True))
    elif dist == 'sqeuclidean':
        positive_distance = K.sum(positive_distance, axis=-1, keepdims=True)
        negative_distance = K.sum(negative_distance, axis=-1, keepdims=True)
    loss = positive_distance - negative_distance
    if margin == 'maxplus':
        loss = K.maximum(0.0, 2 + loss)
    elif margin == 'softplus':
        loss = K.log(1 + K.exp(loss))
        
    returned_loss = K.mean(loss)
    return returned_loss

In [4]:
model = ResNet50(weights='imagenet')

In [5]:
# Remove the last layer (Needed to later be able to create the Siamese Network model)
model.layers.pop()

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
# First freeze all layers of ResNet50. Transfer Learning to be applied.
for layer in model.layers:
    layer.trainable = False
    
# All Batch Normalization layers still need to be trainable so that the "mean"
# and "standard deviation (std)" params can be updated with the new training data
model.get_layer('bn_conv1').trainable = True
model.get_layer('bn2a_branch2a').trainable = True
model.get_layer('bn2a_branch2b').trainable = True
model.get_layer('bn2a_branch2c').trainable = True
model.get_layer('bn2a_branch1').trainable = True
model.get_layer('bn2b_branch2a').trainable = True
model.get_layer('bn2b_branch2b').trainable = True
model.get_layer('bn2b_branch2c').trainable = True
model.get_layer('bn2c_branch2a').trainable = True
model.get_layer('bn2c_branch2b').trainable = True
model.get_layer('bn2c_branch2c').trainable = True
model.get_layer('bn3a_branch2a').trainable = True
model.get_layer('bn3a_branch2b').trainable = True
model.get_layer('bn3a_branch2c').trainable = True
model.get_layer('bn3a_branch1').trainable = True
model.get_layer('bn3b_branch2a').trainable = True
model.get_layer('bn3b_branch2b').trainable = True
model.get_layer('bn3b_branch2c').trainable = True
model.get_layer('bn3c_branch2a').trainable = True
model.get_layer('bn3c_branch2b').trainable = True
model.get_layer('bn3c_branch2c').trainable = True
model.get_layer('bn3d_branch2a').trainable = True
model.get_layer('bn3d_branch2b').trainable = True
model.get_layer('bn3d_branch2c').trainable = True
model.get_layer('bn4a_branch2a').trainable = True
model.get_layer('bn4a_branch2b').trainable = True
model.get_layer('bn4a_branch2c').trainable = True
model.get_layer('bn4a_branch1').trainable = True
model.get_layer('bn4b_branch2a').trainable = True
model.get_layer('bn4b_branch2b').trainable = True
model.get_layer('bn4b_branch2c').trainable = True
model.get_layer('bn4c_branch2a').trainable = True
model.get_layer('bn4c_branch2b').trainable = True
model.get_layer('bn4c_branch2c').trainable = True
model.get_layer('bn4d_branch2a').trainable = True
model.get_layer('bn4d_branch2b').trainable = True
model.get_layer('bn4d_branch2c').trainable = True
model.get_layer('bn4e_branch2a').trainable = True
model.get_layer('bn4e_branch2b').trainable = True
model.get_layer('bn4e_branch2c').trainable = True
model.get_layer('bn4f_branch2a').trainable = True
model.get_layer('bn4f_branch2b').trainable = True
model.get_layer('bn4f_branch2c').trainable = True
model.get_layer('bn5a_branch2a').trainable = True
model.get_layer('bn5a_branch2b').trainable = True
model.get_layer('bn5a_branch2c').trainable = True
model.get_layer('bn5a_branch1').trainable = True
model.get_layer('bn5b_branch2a').trainable = True
model.get_layer('bn5b_branch2b').trainable = True
model.get_layer('bn5b_branch2c').trainable = True
model.get_layer('bn5c_branch2a').trainable = True
model.get_layer('bn5c_branch2b').trainable = True
model.get_layer('bn5c_branch2c').trainable = True


In [8]:
# Used when compiling the siamese network
def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)  # This is actually just returning y_pred bcs
                                        # K.mean has already been called in the triplet_loss function

In [9]:
# Create the siamese network

x = model.get_layer('flatten_1').output # layer 'flatten_1' is the last layer of the model
model_out = Dense(128, activation='relu',  name='model_out')(x)
model_out = Lambda(lambda  x: K.l2_normalize(x,axis=-1))(model_out)

new_model = Model(inputs=model.input, outputs=model_out)

anchor_input = Input(shape=(224, 224, 3), name='anchor_input')
pos_input = Input(shape=(224, 224, 3), name='pos_input')
neg_input = Input(shape=(224, 224, 3), name='neg_input')

encoding_anchor   = new_model(anchor_input)
encoding_pos      = new_model(pos_input)
encoding_neg      = new_model(neg_input)

loss = Lambda(triplet_loss)([encoding_anchor, encoding_pos, encoding_neg])

siamese_network = Model(inputs  = [anchor_input, pos_input, neg_input], 
                        outputs = loss) # Note that the output of the model is the 
                                        # return value from the triplet_loss function above

siamese_network.compile(optimizer=Adam(lr=.00004, clipnorm=1.), loss=identity_loss)

In [10]:
#siamese_network.summary()
#print("bn3d_branch2c: (siamese)\n", 
#      siamese_network.get_layer('model_1').get_layer('bn3d_branch2c').get_weights())

In [11]:
#print(siamese_network.get_layer('model_1').get_layer('res3d_branch2c').get_weights())

In [12]:
train_path1 = '/home/cesncn/Desktop/github_projects/face_recognition/dataset/train/train1.csv'
TRAIN_INPUT_PATHS = [train_path1]

RECORD_DEFAULTS_TRAIN = [[0], [''], [''], ['']]

def decode_csv_train(line):
   parsed_line = tf.decode_csv(line, RECORD_DEFAULTS_TRAIN)
   anchor_path = parsed_line[1]
   pos_path  = parsed_line[2]
   neg_path    = parsed_line[3]
   return anchor_path, pos_path, neg_path

In [13]:
iterator_batch_size = 64
train_batch_size = 8

filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.Dataset.from_tensor_slices(filenames)
dataset = dataset.flat_map(lambda filename: tf.data.TextLineDataset(filename).skip(1).map(decode_csv_train))
dataset = dataset.shuffle(buffer_size=100000)
dataset = dataset.batch(iterator_batch_size)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

train_eval_score = 0

# Use the agnostic tensorflow session from Keras
sess = K.get_session()    

#print("bn3d_branch2c: (Should be same)\n", 
#      siamese_network.get_layer('model_1').get_layer('bn3d_branch2c').get_weights())
#print("res3d_branch2c: (Should be same)\n", 
#      siamese_network.get_layer('model_1').get_layer('res3d_branch2c').get_weights())
print(siamese_network.get_layer('model_1').get_layer('bn3d_branch2c').trainable, "  Should be TRUE")
print(siamese_network.get_layer('model_1').get_layer('res3d_branch2c').trainable, "  Should be FALSE")

nr_epochs = 25   
for i in range(0, nr_epochs):
    print("\nnr_epoch: ", str(i), "\n")
    sess.run(iterator.initializer, feed_dict={filenames: TRAIN_INPUT_PATHS})
    while True:
        try:
          anchor_path, pos_path, neg_path = sess.run(next_element)

          anchor_imgs = np.empty((0, 224, 224, 3))
          pos_imgs = np.empty((0, 224, 224, 3))
          neg_imgs = np.empty((0, 224, 224, 3))
          for j in range (0, len(anchor_path)):
              #print(anchor_path)
              anchor_img = image.load_img(anchor_path[j], target_size=(224, 224))
              anchor_img = image.img_to_array(anchor_img)
              #print(anchor_imgs.shape)
              anchor_img = np.expand_dims(anchor_img, axis=0)
              #print(anchor_imgs.shape)
              anchor_img = preprocess_input(anchor_img)
              anchor_imgs = np.append(anchor_imgs, anchor_img, axis=0)
              #print(anchor_img.shape)

              #print(test_path)
              pos_img = image.load_img(pos_path[j], target_size=(224, 224))
              pos_img = image.img_to_array(pos_img)
              pos_img = np.expand_dims(pos_img, axis=0)
              pos_img = preprocess_input(pos_img)
              pos_imgs = np.append(pos_imgs, pos_img, axis=0)
              #print(pos_img.shape)

              neg_img = image.load_img(neg_path[j], target_size=(224, 224))
              neg_img = image.img_to_array(neg_img)
              neg_img = np.expand_dims(neg_img, axis=0)
              neg_img = preprocess_input(neg_img)
              neg_imgs = np.append(neg_imgs, neg_img, axis=0)
              #print(neg_img.shape)

          #print("len(anchor_imgs): ", len(anchor_imgs))
          #print("pos_imgs[0].shape: ", pos_imgs[0].shape)
          #print("neg_imgs.shape: ", neg_imgs.shape)

          #print(labels)

          # dummy output, needed for being able to run the fit(..) function
          z = np.zeros(len(anchor_path))

          siamese_network.fit(x=[anchor_imgs, pos_imgs, neg_imgs], 
                              y=z, 
                              batch_size=train_batch_size, 
                              epochs=1, 
                              verbose=1, 
                              callbacks=None, 
                              validation_split=0.0, 
                              validation_data=None, 
                              shuffle=True, 
                              class_weight=None, 
                              sample_weight=None, 
                              initial_epoch=0, 
                              steps_per_epoch=None, 
                              validation_steps=None)

        except tf.errors.OutOfRangeError:
          print("Out of range error triggered (looped through training set).")
          break

# Training completed at this point. Save the model architecture and weights.            
            
# Save the Siamese Network architecture
siamese_model_json = siamese_network.to_json()
with open("saved_model/siamese_network_arch.json", "w") as json_file:
    json_file.write(siamese_model_json)
# save the Siamese Network model weights
siamese_network.save_weights('saved_model/siamese_model_weights.h5')

# create and save the Encoding Network to use in predictions later on
encoding_input = Input(shape=(224, 224, 3), name='encoding_input')
encoding_output   = new_model(encoding_input)
encoding_network = Model(inputs  = encoding_input, 
                         outputs = encoding_output)

weights = siamese_network.get_layer('model_1').get_weights()
encoding_network.get_layer('model_1').set_weights(weights)

# Save the Encoding Network architecture
encoding_model_json = encoding_network.to_json()
with open("saved_model/encoding_network_arch.json", "w") as json_file:
    json_file.write(encoding_model_json)
# save the Encoding Network model weights    
encoding_network.save_weights('saved_model/encoding_network_weights.h5')

#print("bn3d_branch2c: (Should be different)\n", 
#      siamese_network.get_layer('model_1').get_layer('bn3d_branch2c').get_weights())
#print("res3d_branch2c: (Should be same)\n", 
#      siamese_network.get_layer('model_1').get_layer('res3d_branch2c').get_weights())

True   Should be TRUE
False   Should be FALSE

nr_epoch:  0 

Epoch 1/1
64/64 [==============================] - 220s 3s/step - loss: 1.8796
Epoch 1/1
64/64 [==============================] - 213s 3s/step - loss: 1.8361
Epoch 1/1
57/57 [==============================] - 201s 4s/step - loss: 1.8120
Out of range error triggered (looped through training set).

nr_epoch:  1 

Epoch 1/1
64/64 [==============================] - 288s 4s/step - loss: 1.8152
Epoch 1/1
64/64 [==============================] - 284s 4s/step - loss: 1.7490
Epoch 1/1
57/57 [==============================] - 270s 5s/step - loss: 1.7618
Out of range error triggered (looped through training set).

nr_epoch:  2 

Epoch 1/1
64/64 [==============================] - 374s 6s/step - loss: 1.7497
Epoch 1/1
64/64 [==============================] - 350s 5s/step - loss: 1.7112
Epoch 1/1
57/57 [==============================] - 248s 4s/step - loss: 1.6987
Out of range error triggered (looped through training set).

nr_epoch:  3 
